In [ ]:
import imaplib
from getpass import getpass

## Creating Server ##

In [ ]:
def server(host_address,port_number):
    global svr
    svr = imaplib.IMAP4_SSL(host = host_address,port = port_number)
server("imap.gmail.com",993) #for gmail

## User Function ##

In [ ]:
def user():
    global Mail_Address , Password
    Mail_Address = input("Enter the Mail_Address: ")
    Password = getpass("Enter Password : ")
    svr.login("{ad}".format(ad = Mail_Address),"{pw}".format(pw = Password))
user()

## Creating ,Renaming ,Deleting Mail Box ##

In [ ]:
def create_mailbox(Mailbox_Name):
    response , details = svr.create(Mailbox_Name)
    print("Create Response : {rs}".format(rs=response))
    print("Create Details : {rd}".format(rd=details))
    
def rename_mailbox(Earlier_Name,New_Name):
    response ,details = svr.rename(Earlier_Name,New_Name)
    print("Rename Response : {rs}".format(rs=response))
    print("Rename Details : {rd}".format(rd=details))

def delete_mailbox(Del_Mailbox):
    response ,details =svr.delete(Del_Mailbox)
    print("Delete Response : {rs}".format(rs=response))
    print("Delete Details : {rd}".format(rd=details))

## Function To Search Specific Mail ##

In [ ]:
def select_search_mail():
    svr.select('INBOX')
    print("Input Search criteria in format (FROM *** SUBJECT ***) OR input ALL")
    search_criteria = input("Specify your search :")
    charset = None 
    
    global Message_Numbers_Data  
    Response , Message_Numbers_data = svr.search(charset, search_criteria)
    #Message_Numbers_data is a list containing number of mails as number in single item(single element in a list) 
    #We need to convert it into a list of separate items for separate mails (like 18 mails , 18 items in list)
    #This we are doing to acccess a particular mail separately
    print("Response : {rp}".format(rp = Response))
    #print("Message Numbers Data : {md}".format(md =Message_Numbers_data))
    
    global Message_Numbers
    Message_Numbers = Message_Numbers_data[0].split()
    print("Message Numbers : {nm}".format(nm = Message_Numbers))
    
    global decoded_Message_Numbers
    decoded_Message_Numbers =[]
    for i in range(len(Message_Numbers)):
        decoded_Message_Numbers.append(int(Message_Numbers[i].decode()))
        
    print("Decoded Message Numbers :{dc}".format(dc =decoded_Message_Numbers))
    
select_search_mail()

## Function To fetch Message ##

In [ ]:
import email #needed for parsing ,decoding ,getting body sub from separately

def fetch_message():
    global Message #Decoded Message
    Message = []
    for a_message_number in Message_Numbers:
        Response , Fetched_message = svr.fetch(a_message_number , "(RFC822)")
        
        #Fetched_message is bytes of code .Hence need to parse.
        #And Fetched message is an 1x2 array ,and the message of bytes is present in the second column so [0][1] 
        
        Decoded_Message = email.message_from_bytes(Fetched_message[0][1])
        Message.append(Decoded_Message)
        
fetch_message()

## Function for Decoding message and Displaying ##

In [ ]:
from email.header import decode_header
import os
import webbrowser

global Date_Data ,From_data , To_data ,Subject_Data, Body_Data 
Date_Data =[]
From_data =[]
To_data = []
Subject_Data =[]
Body_Data =[]
  

In [ ]:
def decode():
    N = int(input("Enter the Number of top Emails you want to open :"))
    html_open = input("Do you want to open the HTML Webpage(Y/N)? : ")
    x = decoded_Message_Numbers[-1]-1
    for i in range(x,x-N,-1) :
        
        Date = Message[i].get("Date")
        Date_Data.append(Date)
        
        From = Message[i].get("From")
        From_data.append(From)
        
        To = Message[i].get("To")
        To_data.append(To)
        
        print("Date : ", Date)
        print("From : ", From)
        print("To :", To)


        Subject = decode_header(Message[i]["Subject"])[0][0] #decode_header returns (decoded_string,charset) ,so for subject we use [0][0]
        if isinstance(Subject,bytes):
            Subject = Subject.decode() #decode convert to strings
        Subject_Data.append(Subject)
        print("Subject : ",Subject)


        if Message[i].is_multipart(): # Multipart is the one which mail contains differnt parts like plain ,html  
            for part in Message[i].walk():  #iterate over the parts
                content_type = part.get_content_type()  #get_content_type and get_content_disposition comes from email package
                content_disposition = str(part.get_content_disposition())
                try:                              # IF THE MESSAGE IS MULTIPART AND DECODE IS TRUE IT WILL RETURN WHICH CAN'T BE FURTHER DECODED ,SO WILL USE TRY
                    body  = part.get_payload(decode = True).decode() #  body = part.get_payload(decode = True) before try you will get None which is not decodable
                except:
                    pass

                if content_type == "text/plain" and "attachment" not in content_disposition:
                    Body_Data.append(body)
                    print(body)
                    
                elif content_type == "text/html" and "attachment" not in content_disposition: 
                    if not os.path.isdir(Subject):
                        os.mkdir(Subject)
                    filename = "{fn}.html".format(fn = Subject[0:40])
                    filepath = os.path.join(Subject,filename)
                    fp = open(filepath,"w",encoding ='utf-8')
                    fp.write(body)
                    fp.close()
                    if html_open.upper() == "Y":
                        webbrowser.open(filepath)
                    elif html_open.upper() == "N":
                        None
                    else :
                        print("Only yes or No.")
                        
                elif "attachment" in content_disposition:
                    file_name = part.get_filename() #gets from content_disposition ,read documentation
                    if file_name:
                        if not os.path.isdir(Subject):
                            os.mkdir(Subject)
                        filepath = os.path.join(Subject , file_name)
                        fp = open(filepath,"wb")
                        fp.write(part.get_payload(decode=True))
                        fp.close()
        
        else:
            content_type = Message[i].get_content_type()
            body = Message.get_payload(decode = True).decode()
            if content_type == "text/plain":
                Body_Data.append(body)
                print(body)
            if content_type == "text/html":   #Cretae a new html file and display in browser
                if not os.path.isdir(Subject):
                    os.mkdir(Subject)
                filename = "{fn}.html".format(fn = Subject[0:40])
                filepath = os.path.join(Subject,filename)
                fp = open(filepath,"w")
                fp.write(body)
                fp.close()
                if html_open.upper() == "Y":
                    webbrowser.open(filepath)
                elif html_open.upper() == "N":
                    None
                else :
                    print("Only yes or No.")

decode()


    

## Function to load in .csv and to remove duplicates ##

In [ ]:
import csv
#csv_file_name even you can create or overwrite as it will elimainted duplicates
def load_csv(csv_file_name):
    global file
    file = open(csv_file_name,"a+",newline ='',encoding = 'utf-8')
    with file :
        header = ['From' , 'To' ,  'Date' , 'Subject' , 'Body']
        writer = csv.DictWriter(file , fieldnames= header)
        writer.writeheader()
        for i in range(0,len(From_data)):
            writer.writerow({'From': From_data[i],
                             'To': To_data[i],
                             'Date':Date_Data[i],
                             'Subject':Subject_Data[i],
                             'Body':Body_Data[i]})
    
    # To remove duplicates in .csv file
    rows = csv.reader(open(csv_file_name,"r",encoding ='utf-8'))
    newrows = []
    for row in rows:
        if row not in newrows:
            newrows.append(row)
    writer = csv.writer(open(csv_file_name,"w",encoding='utf-8'))
    writer.writerows(newrows)
    
#load_csv("csv_file")